In [ ]:
##### import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error

from tensorflow.keras import models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, SimpleRNN, Activation, Dropout, Dense, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
##### definition
def plot_graph(x, y, title, xlabels, ymin, ymax):
    ax.plot(x, y, color='steelblue')
    ax.set_ylim(ymin,ymax)
    ax.set_xlim(-5,150)
    ax.set_xlabel('Year-Month')
    ax.set_ylabel('Averaged delayed time per late flight departure (minutes)')
    ax.xaxis.set_major_locator(MaxNLocator(10))
    plt.xticks(np.arange(0, 145, 12))
    ax.set_xticklabels(xlabels)
    plt.grid(alpha=0.15)
    ax.set_title(title)
    plt.tight_layout()
    plt.show()

def plot_graph_predict(x, y, x2, y2, title, xlabels, ymin, ymax):
    ax.plot(x, y, color='crimson')
    ax.plot(x2, y2, color='steelblue')
    ax.set_ylim(ymin,ymax)
    ax.set_xlim(-5,150)
    ax.set_xlabel('Year-Month')
    ax.set_ylabel('Averaged delayed time per late flight departure (minutes)')
    ax.xaxis.set_major_locator(MaxNLocator(10))
    plt.xticks(np.arange(0, 145, 12))
    ax.set_xticklabels(xlabels)
    plt.grid(alpha=0.15)
    ax.set_title(title)
    plt.tight_layout()
    plt.show()
    
def plot_graph_predict2(x, y, x2, y2, title, xlabels, ymin, ymax):
    ci = 2*np.std(y)
    ax.plot(x2, y2, color='steelblue')
    ax.fill_between(x, (y-ci), (y+ci), color='crimson', alpha=.3)
    ax.set_ylim(ymin,ymax)
    ax.set_xlim(-5,150)
    ax.set_xlabel('Year-Month')
    ax.set_ylabel('Averaged delayed time per late flight departure (minutes)')
    ax.xaxis.set_major_locator(MaxNLocator(10))
    plt.xticks(np.arange(0, 145, 12))
    ax.set_xticklabels(xlabels)
    plt.grid(alpha=0.15)
    ax.set_title(title)
    plt.tight_layout()
    plt.show()
    
def split_sequence(sequence, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequence)):
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out
        if out_end_ix > len(sequence):
            break
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:out_end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

def build_SimpleRNN_model(n_steps_in, n_features):
    model = Sequential()
    model.add(Bidirectional(SimpleRNN(32, activation='tanh', return_sequences=True, input_shape=(n_steps_in, n_features))))
    model.add(Dropout(0.1))

    model.add(Bidirectional(SimpleRNN(64, activation='tanh', return_sequences=True)))
    model.add(Dropout(0.1))

    model.add(Bidirectional(SimpleRNN(128, activation='tanh', return_sequences=True)))
    model.add(Dropout(0.1))

    model.add(Bidirectional(SimpleRNN(256, activation='tanh', return_sequences=True)))
    model.add(Dropout(0.1))

    model.add(Bidirectional(SimpleRNN(1024, activation='tanh', return_sequences=True)))
    model.add(Dropout(0.1))

    model.add(Bidirectional(SimpleRNN(1024, activation='tanh', return_sequences=False)))
    model.add(Dropout(0.1))     
    
    model.add(Dense(512))
    model.add(Dense(units=n_steps_out))
    model.add(Activation("linear"))
    return model

def build_LSTM_model1(n_steps_in, n_features):
    model = Sequential()
    model.add(Bidirectional(LSTM(32, activation='tanh', return_sequences=True, input_shape=(n_steps_in, n_features))))
    model.add(Dropout(0.1))

    model.add(Bidirectional(LSTM(64, activation='tanh', return_sequences=True)))
    model.add(Dropout(0.1))

    model.add(Bidirectional(LSTM(128, activation='tanh', return_sequences=True)))
    model.add(Dropout(0.1))

    model.add(Bidirectional(LSTM(256, activation='tanh', return_sequences=True)))
    model.add(Dropout(0.1))

    model.add(Bidirectional(LSTM(512, activation='tanh', return_sequences=True)))
    model.add(Dropout(0.1))    

    model.add(Bidirectional(LSTM(1024, activation='tanh', return_sequences=False)))
    model.add(Dropout(0.1))
    
    model.add(Dense(512))
    model.add(Dense(units=n_steps_out))
    model.add(Activation("linear"))
    return model

def build_LSTM_model2(n_steps_in, n_features):
    model = Sequential()
    model.add(Bidirectional(LSTM(32, activation='tanh', return_sequences=True, input_shape=(n_steps_in, n_features))))
    model.add(Dropout(0.1))

    model.add(Bidirectional(LSTM(64, activation='tanh', return_sequences=True)))
    model.add(Dropout(0.1))

    model.add(Bidirectional(LSTM(128, activation='tanh', return_sequences=True)))
    model.add(Dropout(0.1))

    model.add(Bidirectional(LSTM(256, activation='tanh', return_sequences=True)))
    model.add(Dropout(0.1))

    model.add(Bidirectional(LSTM(1024, activation='tanh', return_sequences=True)))
    model.add(Dropout(0.1))

    model.add(Bidirectional(LSTM(1024, activation='tanh', return_sequences=False)))
    model.add(Dropout(0.1))     

    model.add(Dense(512))
    model.add(Dense(units=n_steps_out))
    model.add(Activation("linear"))
    return model

In [ ]:
data = pd.read_csv('modified_data.csv')

In [ ]:
dat = data.copy()

In [ ]:
data.info()

In [ ]:
southwest = data[data['OP_CARRIER'] == 'Southwest Airlines']
united = data[data['OP_CARRIER'] == 'United Airlines']
alaska = data[data['OP_CARRIER'] == 'Alaska Airlines']

In [ ]:
sw_delay = southwest.groupby(['year', 'month'])[['DEP_DELAY']].sum()
ua_delay = united.groupby(['year', 'month'])[['DEP_DELAY']].sum()
alaska_delay = alaska.groupby(['year', 'month'])[['DEP_DELAY']].sum()
all_delay = data.groupby(['year', 'month'])[['DEP_DELAY']].sum()

In [ ]:
##### count number of flights per day
sw_count = southwest.groupby(['year', 'month'])[['DEP_DELAY']].count()
ua_count = united.groupby(['year', 'month'])[['DEP_DELAY']].count()
alaska_count = alaska.groupby(['year', 'month'])[['DEP_DELAY']].count()
all_count = data.groupby(['year', 'month'])[['DEP_DELAY']].count()

In [ ]:
##### averaged delayed time per late flight departure
sw = sw_delay/sw_count
ua = ua_delay/ua_count
ak = alaska_delay/alaska_count
total = all_delay/all_count

In [ ]:
s = sw.index.tolist()
sw_index=[]
for i in range(len(s)):
    for j in range(len(s[0])-1):
        date = str(s[i][j])+str('-')+str(s[i][j+1])
        sw_index.append(date)

u = ua.index.tolist()
ua_index=[]
for i in range(len(u)):
    for j in range(len(u[0])-1):
        date = str(u[i][j])+str('-')+str(u[i][j+1])
        ua_index.append(date)
        
a = ak.index.tolist()
ak_index=[]
for i in range(len(a)):
    for j in range(len(a[0])-1):
        date = str(a[i][j])+str('-')+str(a[i][j+1])
        ak_index.append(date)
        
t = total.index.tolist()
total_index=[]
for i in range(len(t)):
    for j in range(len(t[0])-1):
        date = str(t[i][j])+str('-')+str(t[i][j+1])
        total_index.append(date)

In [ ]:
##### data, split into train and test
all_lst = np.array(total['DEP_DELAY'].values.tolist())
train_all = all_lst[:int(len(all_lst)*0.8)]
test_all = all_lst[int(len(all_lst)*0.8):]

swlst = np.array(sw['DEP_DELAY'].values.tolist())
train_sw = swlst[:int(len(swlst)*0.8)]
test_sw = swlst[int(len(swlst)*0.8):]

ualst = np.array(ua['DEP_DELAY'].values.tolist())
train_ua = ualst[:int(len(ualst)*0.8)]
test_ua = ualst[int(len(ualst)*0.8):]

aklst = np.array(ak['DEP_DELAY'].values.tolist())
train_ak = aklst[:int(len(ak)*0.8)]
test_ak = aklst[int(len(ak)*0.8):]

In [ ]:
##### generate x labels
y2019 = ['2019' for i in range(12)]
y2020 = ['2020' for i in range(12)]
y2021 = ['2021' for i in range(2)]
string = y2019+y2020+y2021

mm = np.arange(1,13).tolist()
mm = mm*2
for sublist in ['1','2']:
    mm.append(sublist)

swpredict_index=[]
for i in range(len(mm)):
    date = str(string[i])+str('-')+str(mm[i])
    swpredict_index.append(date)
swpredict_index

predict_sw_index = sw_index + swpredict_index

zero = [0 for i in range(146)]

xlabel=[]
for i in range(0,len(predict_sw_index),12):
    xlabel.append(predict_sw_index[i])
xlabel.append('2021-3')

In [ ]:
##### plot time series
x = total_index
y = all_lst
xlabels = xlabel
ymin = 15
ymax = 50
#title = 'All Airlines, 2009-2018'
title =''
fig, ax = plt.subplots(figsize=(14,6))
plot_graph(x, y, title, xlabels, ymin, ymax)
#fig.savefig('all_origin.png')

In [ ]:
##### plot time series
x = sw_index
y = swlst
ymin = 10
ymax = 45
xlabels = xlabel
#title = 'Southwest Airlines, 2009-2018'
title =''
fig, ax = plt.subplots(figsize=(14,6))
plot_graph(x, y, title, xlabels, ymin, ymax)
#fig.savefig('sw_origin.png')

In [ ]:
##### plot time series
x = ak_index
y = aklst
ymin = 10
ymax = 45
xlabels = xlabel
#title = 'Southwest Airlines, 2009-2018'
title =''
fig, ax = plt.subplots(figsize=(14,6))
plot_graph(x, y, title, xlabels, ymin, ymax)
#fig.savefig('ak_origin.png')

In [ ]:
##### LSTM Model ##
##### Southwest Airlines
raw_seq = swlst[:94]
epoch = 100
learn_rate = 0.0001

##### number of previous steps, number of predicted future steps
n_steps_in, n_steps_out = 66, 24

##### split into samples
X_sw, y_sw = split_sequence(raw_seq, n_steps_in, n_steps_out)
print(X_sw.shape)
print(y_sw.shape)

##### reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X_sw = X_sw.reshape((X_sw.shape[0], X_sw.shape[1], n_features))

callback = ModelCheckpoint(filepath='model00sw.h5', monitor='mse', mode='min', save_best_only=True)

test_ci=[]
future_ci=[]
for i in range(3):
    ##### build model
    model_sw = build_LSTM_model2(n_steps_in, n_features)
    
    ##### compile model
    optimizer = Adam(learning_rate=learn_rate)
    model_sw.compile(optimizer=optimizer, loss='mse', metrics=['mse'])
    
    ##### fit model
    history_sw = model_sw.fit(X_sw, y_sw, epochs=epoch, verbose=1, callbacks=[callback])
    
    ##### predict test data
    x_input_sw = swlst[(94-n_steps_in):94]
    x_input_sw = x_input_sw.reshape((1, n_steps_in, n_features))
    p = model_sw.predict(x_input_sw, verbose=0)
    test_ci.append(p[0])
    
    ##### predict future data
    new_predict_sw = model_sw.predict(np.append(X_sw,y_sw)[24:][np.newaxis,:,np.newaxis])

    ##### append values
    new_pred_sw = np.append(swlst[len(swlst)-1], new_predict_sw)
    future_ci.append(new_pred_sw)

model_sw.summary()

In [ ]:
ci_mean = np.mean(future_ci, axis=0)
ci_test = np.mean(test_ci, axis=0)
new_pred_sw = ci_mean
y_pred_sw = ci_test

In [ ]:
##### calculate mean squared error and mean absolute percentage error
##### southwest
mse_sw = mean_squared_error(swlst[94:], y_pred_sw)
rmse_sw = np.sqrt(mse_sw)
print(rmse_sw)
map_sw = mean_absolute_percentage_error(swlst[94:], y_pred_sw)*100
print(map_sw)

In [ ]:
##### Southwest Airlines
##### plot test prediction
x = np.arange(24)+92
y = y_pred_sw
x2 = np.arange(len(swlst))
y2 = swlst
#x2 = np.arange(95)
#y2 = swlst[:95]
ymin = 10
ymax = 45
xlabels = xlabel
#title = 'Southwest Airlines, 2009-2018'
title =''
fig, ax = plt.subplots(figsize=(14,6))
plot_graph_predict(x, y, x2, y2, title, xlabels, ymin, ymax)
#fig.savefig('sw_predict_test3.png')

##### plot future prediction
x = np.arange(25)+117
y = new_pred_sw
x2 = sw_index
y2 = swlst
ymin = 10
ymax = 45
xlabels = xlabel
#title = 'Southwest Airlines, 2009-2018'
title =''
fig, ax = plt.subplots(figsize=(14,6))
plot_graph_predict2(x, y, x2, y2, title, xlabels, ymin, ymax)
#fig.savefig('sw_predict_future_spread.png')

In [ ]:
##### graph MSE
fig, ax = plt.subplots(figsize=(8,6))
ax.plot(history_sw.history['mse'], color='red')
ax.set_xlabel('Epoch')
ax.set_ylabel('Loss')
plt.legend(['Train'], frameon=False)
plt.show()
#fig.savefig('sw_mse.png')

In [ ]:
##### LSTM Model
##### Alaska Airlines
raw_seq = train_ak
epoch = 20
learn_rate = 0.001

##### number of previous steps, number of predicted future steps
n_steps_in, n_steps_out = 66, 24

##### split into samples
X_ak, y_ak = split_sequence(raw_seq, n_steps_in, n_steps_out)

##### reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X_ak = X_ak.reshape((X_ak.shape[0], X_ak.shape[1], n_features))

#callback = ModelCheckpoint(filepath='model00.h5', monitor='mse', mode='min', save_best_only=True)

test_ci=[]
future_ci=[]
for i in range(3):
    ##### build model
    model_ak = build_LSTM_model1(n_steps_in, n_features)
    
    ##### compile model
    optimizer = Adam(learning_rate=learn_rate)
    model_ak.compile(optimizer=optimizer, loss='mse', metrics=['mse'])
    
    ##### fit model
    #history_alaska = model_sw.fit(X_ak, y_ak, epochs=epoch, verbose=1, callbacks=[callback])
    history_alaska = model_ak.fit(X_ak, y_ak, epochs=epoch, verbose=1)

    ##### predict test data
    x_input_ak = aklst[(94-n_steps_in):94]
    x_input_ak = x_input_ak.reshape((1, n_steps_in, n_features))
    p = model_ak.predict(x_input_ak, verbose=0)
    test_ci.append(p[0])
    
    ##### predict future data
    new_predict_ak = model_ak.predict(np.append(X_ak,y_ak)[24:][np.newaxis,:,np.newaxis])

    ##### append values
    new_pred_ak = np.append(aklst[len(aklst)-1], new_predict_ak)
    future_ci.append(new_pred_ak)

model_ak.summary()

In [ ]:
ci_mean = np.mean(future_ci, axis=0)
ci_test = np.mean(test_ci, axis=0)
new_pred_ak = ci_mean
y_pred_ak = ci_test

In [ ]:
##### calculate mean squared error and mean absolute percentage error
##### southwest
mse_ak = mean_squared_error(aklst[94:], y_pred_ak)
rmse_ak = np.sqrt(mse_ak)
print(rmse_ak)
map_ak = mean_absolute_percentage_error(aklst[94:], y_pred_ak)*100
print(map_ak)

In [ ]:
##### Alaska Airlines
##### plot test prediction
x = np.arange(24)+92
y = y_pred_ak
x2 = np.arange(len(aklst))
y2 = aklst
#x2 = np.arange(95)
#y2 = swlst[:95]
ymin = 10
ymax = 45
xlabels = xlabel
#title = 'Alaska Airlines, 2009-2018'
title =''
fig, ax = plt.subplots(figsize=(14,6))
plot_graph_predict(x, y, x2, y2, title, xlabels, ymin, ymax)
#fig.savefig('ak_predict_test3.png')

##### plot future prediction
x = np.arange(25)+117
y = new_pred_ak
x2 = ak_index
y2 = aklst
ymin = 10
ymax = 45
xlabels = xlabel
#title = 'Alaska Airlines, 2009-2018'
title =''
fig, ax = plt.subplots(figsize=(14,6))
plot_graph_predict2(x, y, x2, y2, title, xlabels, ymin, ymax)
#fig.savefig('ak_predict_future_spread.png')

In [ ]:
##### graph MSE
fig, ax = plt.subplots(figsize=(8,6))
ax.plot(history_alaska.history['mse'], color='red')
ax.set_xlabel('Epoch')
ax.set_ylabel('Loss')
plt.legend(['Train'], frameon=False)
plt.show()
#fig.savefig('ak_mse.png')history_alaska

In [ ]:
##### LSTM Model
##### All Airlines
raw_seq = all_lst[:94]
epoch = 20
learn_rate = 0.001

##### number of previous steps, number of predicted future steps
n_steps_in, n_steps_out = 66, 24

##### split into samples
X_a, y_a = split_sequence(raw_seq, n_steps_in, n_steps_out)

##### reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X_a = X_a.reshape((X_a.shape[0], X_a.shape[1], n_features))

#callback = ModelCheckpoint(filepath='model00.h5', monitor='mse', mode='min', save_best_only=True)

test_ci=[]
future_ci=[]
for i in range(3):
    ##### build model
    model_a = build_LSTM_model1(n_steps_in, n_features)
    
    ##### compile model
    optimizer = Adam(learning_rate=learn_rate)
    model_a.compile(optimizer=optimizer, loss='mse', metrics=['mse'])
    
    ##### fit model
    #history_a = model_sw.fit(X_a, y_a, epochs=epoch, verbose=1, callbacks=[callback])
    history_a = model_a.fit(X_a, y_a, epochs=epoch, verbose=1)

    ##### predict test data
    x_input_a = aklst[(94-n_steps_in):94]
    x_input_a = x_input_a.reshape((1, n_steps_in, n_features))
    p = model_a.predict(x_input_a, verbose=0)
    test_ci.append(p[0])
    
    ##### predict future data
    new_predict_a = model_a.predict(np.append(X_a,y_a)[24:][np.newaxis,:,np.newaxis])

    ##### append values
    new_pred_a = np.append(all_lst[len(all_lst)-1], new_predict_a)
    future_ci.append(new_pred_a)

model_a.summary()

In [ ]:
ci_mean = np.mean(future_ci, axis=0)
ci_test = np.mean(test_ci, axis=0)
new_pred_a = ci_mean
y_pred_a = ci_test

In [ ]:
##### all
mse_a = mean_squared_error(all_lst[94:], ci[2])
rmse_a = np.sqrt(mse_a)
print(rmse_a)
map_a = mean_absolute_percentage_error(all_lst[94:], ci[2])*100
print(map_a)

In [ ]:
##### All Airlines
##### plot test prediction
x = np.arange(24)+92
y = y_pred_a
x2 = np.arange(len(all_lst))
y2 = all_lst
ymin = 15
ymax = 50
xlabels = xlabel
#title = 'All Airlines, 2009-2018'
title =''
fig, ax = plt.subplots(figsize=(14,6))
plot_graph_predict(x, y, x2, y2, title, xlabels, ymin, ymax)
#fig.savefig('all_predict_test3.png')

##### plot future prediction
x = np.arange(25)+117
y = new_pred_a
x2 = total_index
y2 = all_lst
ymin = 15
ymax = 50
xlabels = xlabel
#title = 'All Airlines, 2009-2018'
title =''
fig, ax = plt.subplots(figsize=(14,6))
plot_graph_predict2(x, y, x2, y2, title, xlabels, ymin, ymax)
#fig.savefig('all_predict_future_spread.png')

In [ ]:
##### graph MSE
fig, ax = plt.subplots(figsize=(8,6))
ax.plot(history_a.history['mse'], color='red')
ax.set_xlabel('Epoch')
ax.set_ylabel('Loss')
plt.legend(['Train'], frameon=False)
plt.show()
#fig.savefig('all_mse.png')

In [ ]:
##### load model
#model_sw = models.load_model('model00.h5')
#model_sw.summary()